In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import os,sys
import utilities
import matplotlib.pyplot as plt
from numpy import array
from multiprocessing import Pool
import multiprocessing as mp
from consts import sensor_lat as lat
from consts import sensor_log as log
from consts import valid_sensor_res_10 as valid_sensor_list
from consts import valid_sensor_res_10_old as valid_sensor_list_old
from consts import sensor_day_seconds as day_seconds
from consts import sensor_week_seconds as week_seconds 
from consts import sensor_start_time_unix as start_time
from consts import sensor_end_time_unix as end_time
from consts import sensor_epoch_length as epoch_length
from consts import sensor_optimal_time_interval as optimal_time_interval
from consts import sensor_loc_pic as loc_pic
from consts import sensor_drop_list as drop_list
import consts
import time
import pickle
import copy
import h5py

In [3]:
import torch
import torch.utils.data as Data

In [4]:
time_stamp = np.arange(start_time,end_time+optimal_time_interval,optimal_time_interval)
unix_column = 'Time since the epoch [s]'
idd = 'Station ID'
a_temperature = 'Ambient Temperature'
s_temperature = 'Surface Temperature'
data_index = ['a_temperature_mean','a_temperature_std','a_temperature_array','sub_df_list_optimal_index','time_stamp',]
df_dict = {'rb_optimal':5959,'optimal':5959,'df_list':20160}

In [5]:
with h5py.File('dataset.h5','r') as f:
     a_temperature_mean = f[data_index[0]][:]
     a_temperature_std = f[data_index[1]][:]
     a_temperature_array = f[data_index[2]][:]
     sub_df_list_optimal_index = f[data_index[3]][:]
     time_stamp = f[data_index[4]][:]

In [ ]:
def dataset_write(data_list,key,df_dict=df_dict):
    filename = 'ds_'+key+'.h5'
    store = pd.HDFStore(filename)
    for i,item in enumerate(data_list):
        store[key+'_'+str(i)] = item
    store.close()
    return 1

In [ ]:
def dataset_read(key,df_dict=df_dict):
    filename = 'ds_'+key+'.h5'
    return_list = []
    store = pd.HDFStore(filename)
    for i in range(df_dict[key]):
        return_list.append(store[key+'_'+str(i)])
    store.close()
    return return_list

In [ ]:
rb_sub_df_list_optimal = dataset_read('rb_optimal')

In [7]:
a_temperature_tensor = torch.tensor(a_temperature_array)

In [8]:
a_temperature_tensor.shape

torch.Size([5959, 44])

In [9]:
a_temperature_tensor[0]

tensor([8.2000, 8.8400, 8.7300, 8.7100, 8.1000, 8.5000, 8.3400, 8.8500, 7.9600,
        8.6800, 9.1200, 9.1700, 8.1400, 8.7600, 8.6700, 8.7400, 9.0200, 8.1000,
        8.9000, 8.6800, 9.2900, 8.6600, 8.5800, 8.6000, 8.8800, 8.6500, 9.0000,
        8.3400, 8.8700, 9.0600, 8.9500, 8.6200, 8.8100, 8.9600, 8.2600, 9.0500,
        8.8500, 9.1800, 8.9200, 8.7800, 8.9400, 9.1700, 8.9800, 8.3000],
       dtype=torch.float64)

In [10]:
a_temperature_array.shape

(5959, 44)

In [11]:
index = torch.tensor(np.linspace(1,5959,5959),dtype=int)

In [12]:
torch_dataset = Data.TensorDataset(a_temperature_tensor.float(),index)

In [13]:
train_db,test_db = Data.random_split(a_temperature_tensor.float(),[5000,959])

In [71]:
torch.utils.data.random_split(torch_dataset,[5000,959])

In [14]:
train_loader = Data.DataLoader(
    dataset=train_db,
    batch_size=500,
    shuffle=True,
    num_workers=4
)

In [15]:
test_loader = Data.DataLoader(
    dataset=test_db,
    batch_size=500,
    shuffle=True,
    num_workers=4
)

In [108]:
torch.save(train_loader,'dataloader_32_train.pt')

In [109]:
torch.save(test_loader,'dataloader_32_test.pt')

In [ ]:
a_temperature_tensor.shape

In [ ]:
for (read,_) in loader:
    print(len(read))
    print(read.dtype)

In [ ]:
torch.save(loader,'dataloader_32.pt')

In [16]:
tmp = []
for epoch in range(1):
    for step, (batch,y_index) in enumerate(loader):
        print(type(batch))
        tmp.extend(y_index.numpy())
        print("step:{}, size: {}".format(step,len(batch)))

NameError: name 'loader' is not defined